# ML4CV project work

Summary: Improving and explaining instance segmentation on a litter detection dataset

Members:
- Dell'Olio Domenico
- Delvecchio Giovanni Pio
- Disabato Raffaele

The project was developed in order to improve instance segmentation results on the [TACO Dataset](http://tacodataset.org/).

We decided to implement and test various architectures, among the highest scoring on COCO instance segmentation datasets, in order to compare their performances.
We also tested some explainability methods on these models to try and explain model predictions.

## This notebook contains:
- SOLOv2 model training

In [ ]:
# Installing Detectron2 and required libraries
!pip install 'git+https://github.com/facebookresearch/detectron2.git@5aeb252b194b93dc2879b4ac34bc51a31b5aee13'
!pip install rapidfuzz==2.15.1
!python -m pip install numpy==1.23.1
import numpy as np
np.bool = np.bool_

In [ ]:
# Repository cloning and requirements installation
%cd /content/
!git clone https://github.com/DomMcOyle/TACO-expl.git
%cd /content/TACO-expl
!git checkout solov2
!git pull origin solov2
%cd /content/TACO-expl/AdelaiDet/
!python setup.py build develop

In [ ]:
# loading drive
from google.colab import drive
drive.mount("/content/MyDrive/", force_remount = True)

Mounted at /content/MyDrive/


In [ ]:
# checking detectron2 installation and nvcc version
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.2 ; cuda:  cu121
detectron2: 0.6


In this Notebook we explore the capabilites of the SOLOv2 architecture for instance segmentation presented by Wang et Al. in this [paper](https://arxiv.org/pdf/2003.10152).

![](./res/solov2.png)

The architecture is an improved version of [SOLO](https://arxiv.org/pdf/1912.04488) (Segmenting Objects by LOcations) by the same authors. The original work proposes a one-shot instance segmentation architecture based on the idea of sub-dividing the input in a SxS grid and for each patch the network produces in parallel a classification distribution (for the object in the cell) and a segmentation mask (class-agnostic). Both prediction are conditioned to the position of the object (normalized pixel coordinates are concatenated to the input) and are produced at different scales. In fact, the network uses a backbone network and a FPN to extract features at different scales. Instances are finally selected with NMS.

The second version of this architecture improves the efficiency of the NMS techinique and the time and space efficiency of the network itself. In fact, considering that the object are sparsely distributed on the image, its highly inefficient to produce a mask for all the locations. For this reason, locations are pre-filtered based on their features and then their maps are obtained by convoluting a feature map that condenses information from all the FPN scales and a learned kernel (also conditioned on the location) that avoids differentiating the last convolution layer for each scale.

The framework is pre-implemented in [Detectron 2](https://github.com/facebookresearch/detectron2) and we exploit this library to perform our experiments.

In [ ]:
%cd /content/
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.modeling import build_model
%cd /content/TACO-expl/AdelaiDet/adet/modeling/solov2/
from solov2 import SOLOv2

/content
/content/TACO-expl/AdelaiDet/adet/modeling/solov2


We didn't train the model from scratch, because we opted for a fine-tuning of the weights already provided within the repository.

The chosen backbone for the model was ResNet50.

In [ ]:
import sys

cfg_solo_base_path = '/content/TACO-expl/AdelaiDet/configs/SOLOv2/Base-SOLOv2.yaml'
cfg_solo_r50_path = '/content/TACO-expl/AdelaiDet/configs/SOLOv2/R50_3x.yaml'

The training is handled by a config file in yaml format which has to be modified and loaded beforehand.

In [ ]:
sys.path.append('/content/TACO-expl/AdelaiDet/adet/config')
from defaults import _C

def setup_cfg(cfg_base, cfg_backbone):
  """
  loads config from file and command-line arguments
  :param cfg_base: config file path
  :param cfg_backbone: uconfig file path of the backbone
  """
  cfg = _C
  cfg.merge_from_file(cfg_base)
  cfg.merge_from_file(cfg_backbone)
  return cfg

cfg_solov2 = setup_cfg(cfg_solo_base_path, cfg_solo_r50_path)
print(cfg_solov2)

CUDNN_BENCHMARK: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  FILTER_EMPTY_ANNOTATIONS: True
  NUM_WORKERS: 4
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: ()
  PROPOSAL_FILES_TRAIN: ()
  TEST: ('coco_2017_val',)
  TRAIN: ('coco_2017_train',)
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    CROP_INSTANCE: True
    ENABLED: False
    SIZE: [0.9, 0.9]
    TYPE: relative_range
  FORMAT: BGR
  HFLIP_TRAIN: True
  IS_ROTATE: False
  MASK_FORMAT: bitmask
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (640, 672, 704, 736, 768, 800)
  MIN_SIZE_TRAIN_SAMPLING: choice
  RANDOM_FLIP: horizontal
MODEL:
  ANCHOR_GENERATOR:
    ANGLES: [[-90, 0, 90]]
    ASPECT_RATIOS: [[0.5, 1.0, 2.0]]
    NAME: DefaultAnchorGenerator
    OFFSET: 0.0
    SIZES: [[32, 64, 128, 256, 512]]
  BACKBONE:
    ANTI_ALIAS: False
    FREEZE_AT: 2
    NAME: build_resnet_fpn

We leave the following cell reporting the number of trained parameters for the SOLOv2

In [ ]:
model = build_model(cfg_solov2)
num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(num_trainable_params)

46317392


In [ ]:
# sets the radnom seed
DEFAULT_RANDOM_SEED = 42
# basic random seed
def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
# torch random seed
def seedTorch(seed=DEFAULT_RANDOM_SEED):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
# combine
def seedEverything(seed=DEFAULT_RANDOM_SEED):
    seedBasic(seed)
    seedTorch(seed)

seedEverything()

In [ ]:
#!mkdir /content/official/
!cp -r /content/MyDrive/MyDrive/official/ /content/
!mkdir /content/output/
!cp -r /content/MyDrive/MyDrive/solo_models/chkpt/ /content/output/

We also load and register the dataset, generated in the "MFDETR Training Notebook.ipynb" and modified in "validation resizing.ipynb" as a coco dataset instance.

Differently from the one employed in the Mask-Frozen DETR Training, the images in the validation set and test set are already rotated and resized to a 800x1333 maximum size due to resources requirements. Also the images in the training set are already rotated. This was done to simplify the Detectron2 training pipeline, which is already quite convoluted.

In [ ]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog

with open("/content/TACO-expl/data/annotations_off_0_train.json", "r") as f:
    dataset = json.loads(f.read())
classes = [elem["name"] for elem in dataset["categories"]]

train_annotation_file = '/content/TACO-expl/data/annotations_off_0_train.json'
val_annotation_file = '/content/TACO-expl/data/annotations_off_0_val.json'

img_dir = '/content/official/'


register_coco_instances("TACO_train", {}, train_annotation_file, img_dir)
MetadataCatalog.get("TACO_train").set(thing_classes = classes)
dataset_dicts_train = DatasetCatalog.get("TACO_train")

register_coco_instances("TACO_val", {}, val_annotation_file, img_dir)
MetadataCatalog.get("TACO_val").set(thing_classes = classes)
dataset_dicts_val = DatasetCatalog.get("TACO_val")

[05/10 06:07:46 d2.data.datasets.coco]: Loaded 1200 images in COCO format from /content/TACO-expl/data/annotations_off_0_train.json
[05/10 06:07:46 d2.data.datasets.coco]: Loaded 150 images in COCO format from /content/TACO-expl/data/annotations_off_0_val.json




Here we check some of the configuration values employed for training. The training recipe provided in the repository was used as starting point for the one we actually employed in the end, which consists of:

- Image normalization.
- Data augmentation consisting of Random Horizondtal flip, random resizing with maximum size 1333x800, Random crop.
- Optimizer is AdamW with 1e-4 weight decay and gradient clipping to 1
- Learning rate is set to 1e-3
and scheduled with a Warm-up cosine scheduler. The minimum learning rate was 1e-4.
- Batch size set to 8.
- Maximum Number of epochs set to 12.

In [ ]:
train_cfg = cfg_solov2.clone()
with open("/content/TACO-expl/taco_train_solov2.yaml", "w") as f:
  f.write(train_cfg.dump())

In [ ]:
train_cfg_loaded = get_cfg()
train_cfg_loaded.set_new_allowed(True)
train_cfg_loaded.merge_from_file("/content/TACO-expl/solov2_config/taco_train_solov2.yaml")
print(train_cfg_loaded.SOLVER.IMS_PER_BATCH)
print(train_cfg_loaded.DATALOADER.NUM_WORKERS)
print(train_cfg_loaded.OUTPUT_DIR)
print(train_cfg_loaded.MODEL.WEIGHTS)

8
4
/content/output/chkpt/
/content/output/chkpt/model_0001949.pth


By running the following cells, the model can be fine-tuned. Our experiments on this model were actually pretty scarce as the initial results were quite poor even when compared with those in the TACO paper.

In fact, we tried to change the recipe by modifying learning rate and the number of stages of the backbone to be frozen, but the results were always around 12-13% MAP, so we abandoned the idea of adopting this model, because even if in is indesputably faster than MaskDINO and Mask-Frozen DETR, the tradeoff in performances is too high.

In [ ]:
%cd /content/TACO-expl/AdelaiDet/tools/
from train_net import Trainer
%cd /content/

In [ ]:
trainer = Trainer(train_cfg_loaded)
trainer.build_hooks()

[05/10 06:29:47 d2.engine.defaults]: Model:
SOLOv2(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2): Seque

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


 None,

In [ ]:
trainer.resume_or_load(resume = True)  # load last checkpoint or MODEL.WEIGHTS
trainer.train()

[05/10 07:19:26 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                              | Names in Checkpoint                                                                                  | Shapes                                          |
|:--------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*           | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.bottom_up.res2.0.conv2.*           | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.bottom_up.res2.0.conv3.*           | backbone.bottom_up.res2.0.conv3.{norm.bias,norm.running_mean,

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/10 07:20:08 d2.utils.events]:  eta: 3:27:05  iter: 1959  total_loss: 1.25  loss_ins: 0.9572  loss_cate: 0.2914  time: 3.7122  data_time: 2.2751  lr: 0.00076578  max_mem: 8286M
[05/10 07:21:13 d2.utils.events]:  eta: 2:58:10  iter: 1979  total_loss: 1.426  loss_ins: 1.122  loss_cate: 0.2944  time: 3.3850  data_time: 1.6352  lr: 0.00076145  max_mem: 8419M
[05/10 07:22:24 d2.data.datasets.coco]: Loaded 150 images in COCO format from /content/TACO-expl/data/annotations_off_0_val.json
[05/10 07:22:24 d2.data.build]: Distribution of instances among all 10 categories:
|  category  | #instances   |   category    | #instances   |  category  | #instances   |
|:----------:|:-------------|:-------------:|:-------------|:----------:|:-------------|
|   Bottle   | 50           |  Bottle cap   | 30           |    Can     | 21           |
| Cigarette  | 42           |      Cup      | 21           |    Lid     | 8            |
|   Other    | 135          | Plastic bag.. | 77           |  Pop tab   

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/10 07:22:58 d2.evaluation.evaluator]: Inference done 11/150. Dataloading: 0.0019 s/iter. Inference: 0.2404 s/iter. Eval: 4.2811 s/iter. Total: 4.5234 s/iter. ETA=0:10:28
[05/10 07:23:04 d2.evaluation.evaluator]: Inference done 12/150. Dataloading: 0.0021 s/iter. Inference: 0.2417 s/iter. Eval: 4.4006 s/iter. Total: 4.6453 s/iter. ETA=0:10:41
[05/10 07:23:15 d2.evaluation.evaluator]: Inference done 15/150. Dataloading: 0.0036 s/iter. Inference: 0.2469 s/iter. Eval: 4.1582 s/iter. Total: 4.4099 s/iter. ETA=0:09:55
[05/10 07:23:23 d2.evaluation.evaluator]: Inference done 18/150. Dataloading: 0.0033 s/iter. Inference: 0.2304 s/iter. Eval: 3.7105 s/iter. Total: 3.9452 s/iter. ETA=0:08:40
[05/10 07:23:33 d2.evaluation.evaluator]: Inference done 19/150. Dataloading: 0.0033 s/iter. Inference: 0.2293 s/iter. Eval: 4.1802 s/iter. Total: 4.4139 s/iter. ETA=0:09:38
[05/10 07:23:48 d2.evaluation.evaluator]: Inference done 21/150. Dataloading: 0.0033 s/iter. Inference: 0.2462 s/iter. Eval: 4.558

...

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/10 11:55:56 d2.evaluation.evaluator]: Inference done 11/150. Dataloading: 0.0076 s/iter. Inference: 0.2225 s/iter. Eval: 3.1479 s/iter. Total: 3.3780 s/iter. ETA=0:07:49
[05/10 11:56:06 d2.evaluation.evaluator]: Inference done 15/150. Dataloading: 0.0074 s/iter. Inference: 0.2088 s/iter. Eval: 2.7651 s/iter. Total: 2.9817 s/iter. ETA=0:06:42
[05/10 11:56:14 d2.evaluation.evaluator]: Inference done 19/150. Dataloading: 0.0077 s/iter. Inference: 0.2241 s/iter. Eval: 2.5038 s/iter. Total: 2.7363 s/iter. ETA=0:05:58
[05/10 11:56:20 d2.evaluation.evaluator]: Inference done 21/150. Dataloading: 0.0079 s/iter. Inference: 0.2178 s/iter. Eval: 2.5584 s/iter. Total: 2.7850 s/iter. ETA=0:05:59
[05/10 11:56:27 d2.evaluation.evaluator]: Inference done 23/150. Dataloading: 0.0077 s/iter. Inference: 0.2163 s/iter. Eval: 2.6286 s/iter. Total: 2.8535 s/iter. ETA=0:06:02
[05/10 11:56:37 d2.evaluation.evaluator]: Inference done 25/150. Dataloading: 0.0074 s/iter. Inference: 0.2198 s/iter. Eval: 2.816

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/10 12:01:51 d2.evaluation.evaluator]: Total inference time: 0:06:14.999883 (2.586206 s / iter per device, on 1 devices)
[05/10 12:01:51 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:31 (0.215733 s / iter per device, on 1 devices)
[05/10 12:01:51 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[05/10 12:01:51 d2.evaluation.coco_evaluation]: Saving results to /content/output/chkpt/inference/coco_instances_results.json
[05/10 12:01:51 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
[05/10 12:01:51 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[05/10 12:01:51 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.10 seconds.
[05/10 12:01:51 d2.evaluation.fast_eval_api]: Accumulating evaluation results...
[05/10 12:01:52 d2.evaluation.fast_eval_api]: COCOeval_opt.accumulate() finished in 0.06 seconds.
 

OrderedDict([('bbox',
              {'AP': 0.0,
               'AP50': 0.0,
               'AP75': 0.0,
               'APs': 0.0,
               'APm': 0.0,
               'APl': 0.0,
               'AP-Bottle': 0.0,
               'AP-Bottle cap': 0.0,
               'AP-Can': 0.0,
               'AP-Cigarette': 0.0,
               'AP-Cup': 0.0,
               'AP-Lid': 0.0,
               'AP-Other': 0.0,
               'AP-Plastic bag & wrapper': 0.0,
               'AP-Pop tab': 0.0,
               'AP-Straw': 0.0}),
             ('segm',
              {'AP': 12.38762151415335,
               'AP50': 21.96755226726416,
               'AP75': 10.452759541690543,
               'APs': 0.00825082508250825,
               'APm': 4.278477916381564,
               'APl': 16.71542642024696,
               'AP-Bottle': 27.679730827190497,
               'AP-Bottle cap': 24.59293810272514,
               'AP-Can': 8.017971522106816,
               'AP-Cigarette': 1.595264536529435,
      